In [ ]:
from BeamTestHelpers import plot_resolution_table, plot_resolution_with_pulls, plot_resolutions_per_row, plot_avg_resolution_per_row, preprocess_ranking_data
import pandas as pd
import yaml

## Figure configs

In [ ]:
with open("./board_configs_yaml/CERN_Irrad_2025.yaml", "r") as file:

    fig_configs = yaml.safe_load(file)

print(fig_configs.keys())

given_run = 'run4'
selected_fig_config = fig_configs[given_run]

for id in selected_fig_config.keys():
    selected_fig_config[id]['title'] = f"{selected_fig_config[id]['name']} HV{selected_fig_config[id]['HV']}V OS:{selected_fig_config[id]['offset']}"

roles = {}
for board_id, board_info in selected_fig_config.items():
    roles[board_info.get('role')] = board_id

for key, val in selected_fig_config.items():
    print(key, val)

In [ ]:
## resolution_df: time resolution dataframe
resolution_df = pd.read_csv(f'./beam_test_results/CERN_IRRad_2025/T1_OS20_m25C_HV220_0xF_Nov2_12h_Parkedm6_AfterIV/resolution_T1_OS20_m25C_HV220_0xF_Nov2_12h_Parkedm6_AfterIV.csv')

## nevt_track_df: number of events dataframe
nevt_track_df = pd.read_csv(f'./beam_test_results/CERN_IRRad_2025/T1_OS20_m25C_HV220_0xF_Nov2_12h_Parkedm6_AfterIV/T1_OS20_m25C_HV220_0xF_Nov2_12h_Parkedm6_AfterIV_nevt_per_track.csv')

In [ ]:
if nevt_track_df.columns.size == 9:
    col_to_merge = ['row_trig', 'col_trig', 'row_dut', 'col_dut', 'row_extra', 'col_extra', 'row_ref', 'col_ref']
else:
    col_to_merge = ['row_trig', 'col_trig', 'row_dut', 'col_dut', 'row_ref', 'col_ref']

merged_df = pd.merge(resolution_df, nevt_track_df, on=col_to_merge)
merged_df = merged_df.loc[merged_df['err_dut'] != 0]
merged_df.sort_values(by=['nevt'], ascending=False).reset_index(drop=True)

## Define condition by track pixel combinations from merged_df

In [ ]:
condition = (merged_df['row_trig'] == merged_df['row_dut']) & \
            (merged_df['col_trig'] == merged_df['col_dut']-1) & \
            (merged_df['row_trig'] == merged_df['row_ref']) & \
            (merged_df['col_trig'] == merged_df['col_ref']) & \
            (merged_df['row_trig'] == merged_df['row_extra']) & \
            (merged_df['col_trig'] == merged_df['col_extra']-1)

# condition = (merged_df['row_trig'] == merged_df['row_dut']) & \
#             (merged_df['col_trig'] == merged_df['col_dut']) & \
#             (merged_df['row_trig'] == merged_df['row_ref']) & \
#             (merged_df['col_trig'] == merged_df['col_ref']) & \
#             (merged_df['row_trig'] == merged_df['row_extra']) & \
#             (merged_df['col_trig'] == merged_df['col_extra'])


selected_data = merged_df[condition].reset_index(drop=True)
selected_data

In [ ]:
plot_resolution_table(input_df=selected_data, tb_loc='cern', fig_config=selected_fig_config,
                      min_resolution=10, max_resolution=110, slides_friendly = False, show_number=True)

In [ ]:
for id in selected_fig_config.keys():
    selected_fig_config[id]['title'] = f"{selected_fig_config[id]['name']}\nHV{selected_fig_config[id]['HV']}V OS:{selected_fig_config[id]['offset']}"

plot_resolution_with_pulls(input_df=selected_data, list_of_boards=['extra', 'ref', 'dut'],
                           tb_loc='cern', fig_config=selected_fig_config, hist_range=[5, 75], hist_bins=21, print_fit_results=False)

# plot_resolution_with_pulls(input_df=selected_data, list_of_boards=['dut'],
#                            tb_loc='cern', fig_config=selected_fig_config, hist_range=[35, 85], hist_bins=31, print_fit_results=False)

### New 2D map, based on nevt selection

In [ ]:
ranked_df, _, _ = preprocess_ranking_data(merged_df, 'row_dut', 'col_dut')
dut_only_df = ranked_df.loc[(ranked_df['category'] == 'Path 1')][['row_dut', 'col_dut', 'res_dut', 'err_dut', 'nevt']].reset_index(drop=True)
plot_resolution_table(input_df=dut_only_df, tb_loc='irrad', fig_config=selected_fig_config,
                      min_resolution=10, max_resolution=110, slides_friendly = False, show_number=True)

In [ ]:
ranked_df, _, _ = preprocess_ranking_data(merged_df, 'row_ref', 'col_ref')
ref_only_df = ranked_df.loc[(ranked_df['category'] == 'Path 1')][['row_ref', 'col_ref', 'res_ref', 'err_ref', 'nevt']].reset_index(drop=True)
plot_resolution_table(input_df=ref_only_df, tb_loc='irrad', fig_config=selected_fig_config,
                      min_resolution=10, max_resolution=110, slides_friendly = False, show_number=True)

In [ ]:
ranked_df, _, _ = preprocess_ranking_data(merged_df, 'row_extra', 'col_extra')
ref_only_df = ranked_df.loc[(ranked_df['category'] == 'Path 1')][['row_extra', 'col_extra', 'res_extra', 'err_extra', 'nevt']].reset_index(drop=True)
plot_resolution_table(input_df=ref_only_df, tb_loc='irrad', fig_config=selected_fig_config,
                      min_resolution=10, max_resolution=110, slides_friendly = False, show_number=True)

### Make new plots per row

In [ ]:
plot_resolutions_per_row(merged_df, board_role='dut', tb_loc='irrad', fig_config=selected_fig_config, rows_to_draw=[0, 15])
plot_resolutions_per_row(merged_df, board_role='ref', tb_loc='irrad', fig_config=selected_fig_config, rows_to_draw=[0, 15])
plot_resolutions_per_row(merged_df, board_role='extra', tb_loc='irrad', fig_config=selected_fig_config, rows_to_draw=[0, 15])

In [ ]:
plot_avg_resolution_per_row(merged_df, board_role='dut', tb_loc='irrad', fig_config=selected_fig_config)
plot_avg_resolution_per_row(merged_df, board_role='ref', tb_loc='irrad', fig_config=selected_fig_config)
plot_avg_resolution_per_row(merged_df, board_role='extra', tb_loc='irrad', fig_config=selected_fig_config)